# NSSP project 1

In [1]:
%gui wx
import sys
import os

ERROR:tornado.application:Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x106e5e160>)
Traceback (most recent call last):
  File "/Users/djay/miniforge3/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/Users/djay/miniforge3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 449, in advance_eventloop
    eventloop(self)
  File "/Users/djay/miniforge3/lib/python3.9/site-packages/ipykernel/eventloops.py", line 167, in loop_wx
    import wx
ModuleNotFoundError: No module named 'wx'
